In [1]:
# import ee and geemap define map
import ee 
import geemap
geemap.ee_initialize()
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
#*
 # Function to mask clouds using the Sentinel-2 QA band
 # @param {ee.Image} image Sentinel-2 image
 # @return {ee.Image} cloud masked Sentinel-2 image
 #
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                  .filterDate('2022-01-01', '2022-12-31') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                  .map(maskS2clouds)

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(-99.33, 31.05, 5)

Map.addLayer(dataset.mean(), visualization, 'RGB')
Map

Map(center=[31.05, -99.33], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [4]:
# add tiger TX
Texas = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Texas'))
style = {'color': 'white', 'fillColor': '00008000'}
Map.addLayer(Texas.style(**style), {}, 'Tejas')

In [5]:
# Define a function to clip an image to a feature collection
def clipToFeature(image):
  return image.clipToCollection(Texas)
  # Map the clipping function over the ImageCollection
Txclip = dataset.map(clipToFeature)

In [7]:
Map.addLayer(dataset.mean(), visualization, 'RGB', False)
Map.addLayer(Txclip, visualization, 'Texas Sentinel-2')
Map

Map(bottom=3652.0, center=[31.05, -99.33], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…